In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.image as mpimg
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.preprocessing import image
from sklearn.metrics import classification_report, confusion_matrix
import time
from IPython.display import display
from sklearn.model_selection import train_test_split
import tqdm
import random
from keras.preprocessing.image import load_img
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tensorflow.keras.applications import ResNet50V2
from torch.utils.data import Dataset

In [ ]:
#folder path
folder_path = "Downloads/kagglecatsanddogs_5340/Images"

In [ ]:
# Create a list to hold file paths and labels
image_paths = []
labels = []

# Load the paths and labels
for label in ["Cat", "Dog"]:
    label_path = os.path.join(folder_path, "PetImages", label)
    for image in os.listdir(label_path):
        # Ensure the file is an image
        if image.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join("PetImages", label, image)
            image_paths.append(image_path)
            labels.append(1 if label == "Dog" else 0)

# Create a DataFrame
df = pd.DataFrame({
    "image": image_paths,
    "label": labels
})

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
# Create a count plot
sns.countplot(x='label', data=df)
plt.title('Count of Cats and Dogs')
plt.xticks(ticks=[0, 1], labels=['Cats', 'Dogs'])
plt.show()

In [ ]:
# Function to get image resolution
def get_resolution(image_path):
    with Image.open(os.path.join(folder_path, image_path)) as img:
        return img.size


# Apply the function to the first few images
df['resolution'] = df['image'].apply(get_resolution)
df.head()

In [ ]:
df.tail()

In [ ]:

# Function to display images
def display_images(image_paths, n=5):
    plt.figure(figsize=(20, 10))
    for i, image_path in enumerate(image_paths[0:n]):
        full_path = os.path.join(folder_path, image_path)
        img = mpimg.imread(full_path)
        plt.subplot(1, n, i+1)
        plt.imshow(img)
        plt.axis('off')
    plt.show()


# Display first few images
display_images(df['image'])

In [ ]:
#Data Preprocessing

In [ ]:
df['label']=df.label.astype('str')

In [ ]:
df.info()

In [ ]:
base_dir = 'Downloads/kagglecatsanddogs_5340/Images'

# Split data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train_df.shape

In [ ]:
test_df.shape

In [ ]:
# Image augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Flow data from DataFrame
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=base_dir,
    x_col='image',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

In [ ]:
validation_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=base_dir,
    x_col='image',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

In [ ]:
#RESNET

In [ ]:
# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False)

# Add new layers for binary classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [ ]:
# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Freeze the base_model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
#Train Model
start_time = time.time()

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator))

training_time = time.time() - start_time
print(f"Training Time: {training_time}s")

In [ ]:
# Generate predictions
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=base_dir,
    x_col='image',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

predictions = model.predict(test_generator)
predictions = [1 if x > 0.5 else 0 for x in predictions]

# Generate classification report
report = classification_report(
    test_generator.classes, predictions, target_names=['Cat', 'Dog'])
print(report)

In [ ]:
# Plot the training and validation losses
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
def predict_image(img_path,model_clf):
    img = image.load_img(img_path, target_size=(150, 150))
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.

    prediction = model_clf.predict(img_tensor)
    return 'Dog' if prediction[0][0] > 0.5 else 'Cat'

In [ ]:
# Made predictions
predict_image('Downloads/kagglecatsanddogs_5340/test.jpg',model)

In [ ]:
#RESNETV2

In [ ]:
# Load the pre-trained ResNet50V2 model
base_model = ResNet50V2(weights='imagenet', include_top=False)

# Add new layers for binary classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [ ]:
# Define the model
model_v2 = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base_model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile the model
model_v2.compile(optimizer='adam', loss='binary_crossentropy',
                 metrics=['accuracy'])

In [ ]:
start_time = time.time()

history_v2 = model_v2.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator))

In [ ]:
training_time_v2 = time.time() - start_time
print(f"Training Time for ResNetV2: {training_time_v2}s")

In [ ]:
predictions_v2 = model_v2.predict(test_generator)
predictions_v2 = [1 if x > 0.5 else 0 for x in predictions_v2]

# Generate classification report
report_v2 = classification_report(
    test_generator.classes, predictions_v2, target_names=['Cat', 'Dog'])
print(report_v2)

In [ ]:
plt.plot(history_v2.history['loss'], label='train')
plt.plot(history_v2.history['val_loss'], label='validation')
plt.title('Model loss for ResNetV2')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
predict_image(
    'Downloads/kagglecatsanddogs_5340/test.jpg', model_v2)

In [ ]:
#RESNEXT

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        """
        Args:
            dataframe (pandas.DataFrame): DataFrame with two columns 'image' and 'label'.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.df = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.df.iloc[idx, 0])
        image = Image.open(img_name)
        label = self.df.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
from torchvision import transforms

# Modify your transform to include a conversion to RGB
transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert(
        'RGB') if img.mode != 'RGB' else img),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
df2=df.copy()

In [ ]:
df2.label=df2.label.astype('int')
df2.info()

In [ ]:
# Splitting the DataFrame into train and test sets
train_df, test_df = train_test_split(df2, test_size=0.2)

# Instantiate the custom dataset
root_dir = 'Downloads/kagglecatsanddogs_5340/Images'

train_dataset = CustomImageDataset(
    dataframe=train_df, root_dir=root_dir, transform=transform)
test_dataset = CustomImageDataset(
    dataframe=test_df, root_dir=root_dir, transform=transform)


# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
# Load the pre-trained ResNeXt model
model_resnext = models.resnext50_32x4d(pretrained=True)

# final fully connected layer for binary classification
num_ftrs = model_resnext.fc.in_features
model_resnext.fc = nn.Linear(num_ftrs, 2)

# Define the loss function and optimizer
criterion_resnext = nn.CrossEntropyLoss()
optimizer_resnext = torch.optim.Adam(model_resnext.parameters(), lr=0.001)

# Check if GPU is available and move the model to GPU if it is
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_resnext.to(device)

In [ ]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    model.to(device)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch in train_loader:
            inputs = batch[0].to(device)
            labels = torch.tensor(batch[1]).to(
                device)  # Convert labels to a tensor

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(
            f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {total_loss / len(train_loader)}')

        # Validation loop
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                inputs = batch[0].to(device)
                labels = torch.tensor(batch[1]).to(
                    device)  # Convert labels to a tensor

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                total_val_loss += loss.item()

        print(f'Validation Loss: {total_val_loss / len(val_loader)}')

In [ ]:
# Train the model
train_model(model_resnext, criterion_resnext,
            optimizer_resnext, train_loader, test_loader)

In [ ]:
# Revised checks for the DataLoader output format
for data in train_loader:
    print("Data batch structure:", type(data), len(data))
    print("Images structure:", type(data[0]), data[0].shape)
    labels = data[1]
    if isinstance(labels, torch.Tensor):
        print("Labels structure: Tensor", labels.shape)
    else:
        print("Labels structure:", type(labels))
    break  # We only want to check the first batch

In [ ]:
def evaluate_model(model, val_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print(classification_report(all_labels,
          all_preds, target_names=['Cat', 'Dog']))


# Call the evaluate_model function after training
evaluate_model(model_resnext, test_loader)

In [ ]:
#Tensor flow/Keras model parameters
model.summary()

In [ ]:
# Tensor flow/Keras model depth
depth = len(model.layers)
print("Depth of the model (number of layers):", depth)

In [ ]:
# Tensor flow/Keras model parameters
model_v2.summary()

In [ ]:
# Tensor flow/Keras model depth
depth = len(model_v2.layers)
print("Depth of the model (number of layers):", depth)

In [ ]:
#Pytorch model Parameters
total_params = sum(p.numel()
                   for p in model_resnext.parameters() if p.requires_grad)
print("Total number of trainable parameters:", total_params)

In [ ]:
#Pytorch model depth
print(model_resnext)

In [ ]:

# Data from the models' performance
data = {
    "Model": ["ResNet", "ResNetV2", "ResNeXt"],
    "Precision": [0.68, 0.97, 0.91],
    "Recall": [0.62, 0.97, 0.91],
    "F1-Score": [0.57, 0.97, 0.91],
    "Accuracy": [0.61, 0.97, 0.91]
}

# Creating a DataFrame
df = pd.DataFrame(data)

# Setting the plot style
sns.set(style="whitegrid")

# Creating a grouped bar chart
plt.figure(figsize=(10, 6))
tidy = df.melt(id_vars='Model').rename(columns=str.title)
sns.barplot(x='Model', y='Value', hue='Variable', data=tidy)

# Adding labels and title
plt.xlabel('Model', fontsize=14)
plt.ylabel('Score', fontsize=14)
plt.title('Evaluation Metrics for Each Model', fontsize=16)
plt.legend(title='Metric')

# Displaying the plot
plt.show()